In [1]:
import numpy as np
import plotly.express as px
from dubious import Normal, LogNormal, Uniform, Beta, Uncertain, sample_uncertain, Context
rng = np.random.Generator(np.random.PCG64())
ctx = Context()

In [2]:
length_dist = Normal(10, 1)
length = Uncertain(length_dist, ctx=ctx)

# Width is about 5 ± 0.5
width_dist = Normal(5, 0.5)
width = Uncertain(width_dist, ctx=ctx)

In [3]:
area = length * width

# Inspect the uncertainty
print("Mean area:", area.mean())
print("Variance:", area.var())
print("Some samples:", area.sample(5))

Mean area: 50.02432517557902
Variance: 50.63338202302161
Some samples: [52.45935809 55.77334354 58.24078412 46.96891345 41.33346954]


In [4]:

data = area.sample(200000)
fig = px.histogram(
    data,
    nbins=200,
)

fig.show()

In [5]:
print(area.cdf(area.quantile(0.5)))

0.498595
